In [2]:
from imutils import contours
import imutils as im
import cv2 as cv
import numpy as np

In [3]:
def transformation(image, points): #This function takes 4 corner points, orders them and fixes the perspective
    rect = np.zeros((4, 2), dtype='float32')
    s = points.sum(axis=1)
    
    rect[0] = points[np.argmin(s)] = tl
    rect[2] = points[np.argmax(s)] = br
    diff = np.diff(points, axis=1)
    rect[1] = points[np.argmin(diff)] = tr
    rect[3] = points[np.argmax(diff)] = bl

    widthA = np.sqrt(((br[0] - bl[0])**2) + ((br[1]) - bl[1])**2)
    widthB = np.sqrt(((tr[0] - tl[0])**2) + ((tr[1] - tl[1])**2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0])**2) + ((tr[1] - br[1])**2))
    heightB = np.sqrt(((tl[0] - bl[0])**2) + ((tl[1] - bl[1])**2))
    maxHeight = max(int(heightA), int(heightB))
    
    dest = np.array([
        [0,0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32"
    )
    matrix = cv.getPerspectiveTransform(rect, dest)
    trans = cv.warpPerspective(image, matrix, (maxWidth, maxHeight))
    return trans

In [4]:
def extractMinMax(points): #This function fixes the issue of finding more then enough points by averaging them
    tupleSorted = []
    for i in points:
        xmin, ymin = np.min(i, axis=0)[0]
        xmax, ymax = np.max(i, axis=0)[0]
        tupleSorted.append(np.array([[[xmin, ymin]], [[xmax, ymax]]], dtype=np.int32))
    return tuple(tupleSorted)

In [5]:
digitDisplay = { #This stores the way of how every digit is represented
    (1, 1, 1, 0, 1, 1, 1): 0,
    (0, 0, 1, 0, 0, 1, 0): 1,
    (1, 0, 1, 1, 1, 0, 1): 2,
    (1, 0, 1, 1, 0, 1, 1): 3,
    (0, 1, 1, 1, 0, 1, 0): 4,
    (1, 1, 0, 1, 0, 1, 1): 5,
    (1, 1, 0, 1, 1, 1, 1): 6,
    (1, 0, 1, 0, 0, 1, 0): 7,
    (1, 1, 1, 1, 1, 1, 1): 8,
    (1, 1, 1, 1, 0, 1, 1): 9,
}

In [6]:
corn = np.array([ #The input variable for 4 points of display
    [0,0],
    [0,2],
    [2,2],
    [2,0]], dtype = "float32")

In [ ]:
image = output = cv.imread("display.jpg")
image = transformation(image, corn)
hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

lowerColor = np.array([0, 100, 100], dtype=np.uint8) #HSV values in square brackets of lower and higher thresh color
upperColor = np.array([10, 255, 255], dtype=np.uint8)
mask = cv.inRange(hsv, lowerColor, upperColor)
result = cv.bitwise_and(image, image, mask=mask)
gray = cv.cvtColor(result, cv.COLOR_BGR2GRAY)
_, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
thresh = cv.threshold(binary, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
lut = cv.getStructuringElement(cv.MORPH_ELLIPSE, (1, 5)) #In case of segments not connecting, slowly raise second value in brackets
thresh = cv.morphologyEx(thresh, cv.MORPH_OPEN, lut)
thresh = cv.bitwise_not(thresh)

In [ ]:
cont = cv.findContours(binary.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
cont = im.grab_contours(cont)                                                   
digitCont = []  

for c in cont:                             
    (x, y, w, h) = cv.boundingRect(c)      
    if w >= 5 and (h >= 20 and h <= 50):   
        digitCont.append(c)
len(digitCont)       #If the value isn't equal to amount of digits, change the values of 'w' or/and 'h' depending on value being bigger or lower
if (len(digitCont[0])!=2):  #If tuple doesn't have only 2 points (mostly), use given functions
    tupleSorted = extractMinMax(digitCont)
    digitCont = tupleSorted

In [ ]:
for i in digitCont: #This function is used only for checking, if right digits are detected; can be skipped
        pt1 = tuple(i[0][0])
        pt2 = tuple(i[1][0])
        cv.rectangle(image, pt1, pt2, (0, 255, 0), 2)

In [ ]:
for c in digitCont:
    (x, y, w, h) = cv.boundingRect(c)
    roi = thresh[y:y + h, x:x + w]
    (roiH, roiW) = roi.shape
    (sW, sH) = (int(roiW * 0.3), int(roiH * 0.2)) #Float values usually depends on digit on image resolution; needs to be optimise for certain image
    sHC = int(roiH*0.1)
    segments = [
        ((0, 0), (w, sH)),
        ((0, 0), (sW, h // 2)),
        ((w - sW, 0), (w, h // 2)),
        ((0, (h // 2) - sHC) , (w, (h // 2) + sHC)),
        ((0, h // 2), (sW, h)),
        ((w - sW, h // 2), (w, h)),
        ((0, h - sH), (w, h))
    ]
    on = [0] * len(segments) 
    
    for (i, ((xA, yA), (xB, yB))) in enumerate(segments):
        segROI = roi[yA:yB, xA:xB]
        total = cv.countNonZero(segROI) 
        area = (xB - xA) * (yB - yA) 
        if total / float(area) > 0.5: #Depends on how well thresholding was made; if its not reading, lower the value, so x<1
            on[i]= 1

    digit = digitDisplay[tuple(on)]
    digits.append(digit)
    cv.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 1)
    cv.putText(output, str(digit), (x + w + 1, y + h + 1), cv.FONT_HERSHEY_COMPLEX_SMALL, 0.5, (0, 255, 0), 1)

In [ ]:
print(u"{}{}:{}{}".format(*digits))
cv.imshow("Output", output)
cv.imwrite("Result.jpg", output)
cv.waitKey(0)